# Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
from IPython.display import Image
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings
import tensorflow as tf
import tensorflow_datasets as tfds
warnings.filterwarnings("ignore")

# 16.1 순차 데이터 소개

## 16.1.1 순차 데이터 모델링

다른 데이터와 다르게 시퀀스 데이터는 특별하다.

시퀀스 원소들은 특정 순서가 있으므로 상호 독립적이지 않다.

일반적으로 지도 학습을 위한 머신 러닝 알고리즘은 입력 데이터가 독립 동일 분포라고 가정한다.

즉, 훈련 샘플이 상호 독립적이고 같은 분포에 속한다는 의미이다.

이러한 것들은 훈련 샘플의 순서가 관계가 없다.

하지만 시퀀스를 다룰 때는 이런 가정이 유효하지 않다.

시퀀스의 정의가 순서를 고려하기 때문이다. 

예를 들어 주가 데이터.

시계열 데이터 또한 순차 데이터의 한 종류이다.

하지만 모든 데이터가 시간 차원을 갖는 건 아니다.

## 16.1.2 시퀀스 표현

In [3]:
Image(url='https://git.io/JLdVm', width=700)

MLP나 CNN은 훈련 샘플이 서로 독립적이어서 순서 정보와 연관이 없다고 가정한다.

이전에 본 훈련 샘플을 기억하는 메모리가 없다.

예를 들어, 샘플이 정방향 계산과 역전파 단계를 통과하면 가중치는 훈련 샘플의 처리 순서에 상관없이 독립적으로 업데이트된다.

## 16.1.3 시퀀스 모델링의 종류

시퀀스 모델링에는 언어 번역,이미지 캡셔닝, 텍스트 생성과 같은 매력적인 어플리케이션이 많다.

In [4]:
Image(url='https://git.io/JLdVO', width=700)

- 다대일 : 입력 데이터가 시퀀스이지만 출력은 시퀀스가 아닌 고정 크기의 벡터나 스칼라.

  예를 들어 감성분석.

- 일대다 : 입력 데이터가 시퀀스가 아니라 일반적인 형태고 출력은 시퀀스. 예를 들어 이미지 캡셔닝. 입력이 이미지 출력이 이미지 내용을 요약한 문장

- 다대다 : 입력과 출력이 모두 시퀀스. 동기적인 다대다 모델링의 작업은 비디오 분류가 있고 지연이 있는 다대다 모델의 예는 번역이 있다.

# 16.2 시퀀스 모델링을 위한 RNN

## 16.2.1 RNN 반복 구조 이해

In [5]:
Image(url='https://git.io/JLdV3', width=700)

기본 피드포워드 네트워크에서 정보는 입력에서 은닉층으로 흐른 후 은닉층에서 출력층으로 전달된다. 

반면 순환 네트워크에서는 은닉층이 현재 타임 스텝의 입력층과 이전 타임 스텝의 은닉층으로부터 정보를 받는다.

인접한 타임 스텝의 정보가 은닉층의 흐르기 때문에 네트워크가 이전 이벤트를 기억할 수 있다.

이 정보 흐름을 보통 루프라고 표현하는데 그래프 표기법에서는 순환 에지라고도 하기에 RNN 구조 이름이 여기서 유래됐다.

RNN은 여러 개의 은닉층으로 구성할 수 있다. 

In [6]:
Image(url='https://git.io/JLdVs', width=700)

일반 신경망의 은닉 유닛은 입력층에 연결된 최종 입력 하나만 받는다. 

반면 RNN의 유닛은 두 개의 다른 입력을 받는다.

입력층으로부터 받은 입력과 같은 은닉층에서 t-1 타임 스텝의 활성화 출력을 받는다.

맨 처음에는 은닉 유닛이 0 또는 작은 난수로 초기화 된다.

각 은닉층은 시퀀스를 입력으로 받기에 마지막을 제외하고 모든 순환 층은 시퀀스를 출력으로 반환해야 한다.

마지막 순환 층의 동작은 문제 유형에 따라 결정된다.

## 16.2.2 RNN의 활성화 출력 계산

In [7]:
Image(url='https://git.io/JLdVC', width=700)

In [8]:
Image(url='https://git.io/JLdVW', width=700)

## 16.2.3 은닉 순환과 출력 순환

지금까지 은닉층에 순환 성질이 있는 순환 신경망을 보았다.

하지만 출력층에서 오는 순환 연결을 가진 모델도 있다.

In [9]:
Image(url='https://git.io/JLdV8', width=700)

출력-출력 순환

In [10]:
tf.random.set_seed(1)
rnn_layer=tf.keras.layers.SimpleRNN(
    units=2,use_bias=True,return_sequences=True
)

rnn_layer.build(input_shape=(None,None,5))

w_xh,w_oo,b_h=rnn_layer.weights

print('W_xh 크기:', w_xh.shape)
print('W_oo 크기:', w_oo.shape)
print('b_h 크기:', b_h.shape)

W_xh 크기: (5, 2)
W_oo 크기: (2, 2)
b_h 크기: (2,)


In [11]:
x_seq=tf.convert_to_tensor(
    [[1.0]*5,[2.0]*5,[3.0]*5],
    dtype=tf.float32
)

x_seq

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3.]], dtype=float32)>

In [12]:
## SimepleRNN의 출력:
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))

## 수동으로 출력 계산하기:
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('타임 스텝 {} =>'.format(t))
    print('   입력           :', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h    
    print('   은닉           :', ht.numpy())
    
    if t>0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
        
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('   출력 (수동)     :', ot.numpy())
    print('   SimpleRNN 출력 :'.format(t), output[0][t].numpy())
    print()

타임 스텝 0 =>
   입력           : [[1. 1. 1. 1. 1.]]
   은닉           : [[0.41464037 0.96012145]]
   출력 (수동)     : [[0.39240566 0.74433106]]
   SimpleRNN 출력 : [0.39240566 0.74433106]

타임 스텝 1 =>
   입력           : [[2. 2. 2. 2. 2.]]
   은닉           : [[0.82928073 1.9202429 ]]
   출력 (수동)     : [[0.80116504 0.99129474]]
   SimpleRNN 출력 : [0.80116504 0.99129474]

타임 스텝 2 =>
   입력           : [[3. 3. 3. 3. 3.]]
   은닉           : [[1.243921  2.8803642]]
   출력 (수동)     : [[0.95468265 0.99930704]]
   SimpleRNN 출력 : [0.95468265 0.99930704]



## 16.2.4 긴 시퀀스 학습의 어려움

BPTT는 새로운 문제를 야기시킨다.

손실 함수의 기울기를 계산할 때 기울기 폭주, 기울기 소실 문제가 발생한다.

In [13]:
Image(url='https://git.io/JLdV4', width=700)

기본적으로 미분은 t-k개의 곱셈으로 이루어진다. 

즉 w가 t-k번 곱해진다.

기울기 소실이나 폭주를 피하는 단순한 방법은 w가 1이 되도록 하는 것이다.

세 가지 해결책은 다음과 같다.

1. 그레이디언트 클리핑

2. TBPTT(Truncated BackPropagation Through Time)

3. LSTM ( Long Shor-Term Memory)

클리핑을 사용하면 그레이디언트의 임계 값을 지정하고 이 값을 넘어서는 경우 임계 값을 그레이디언트 값으로 사용한다.

TBPTT는 정방향 계산 후 역전파될 수 있는 타임 스텝의 횟수를 제한한다.

두 방법 모두 기울기 폭주 문제를 해결할 수 있지만 기울기가 시간을 거슬러 적절하게 가중치가 업데이트될 수 있는 스텝을 제한한다.

다른 방법으로 호크라이터와 슈미트후버가 고안한 LSTM은 메모리 셀을 활용해 긴 시퀀스를 성공적을 모델링할 수 있었다.

## 16.2.5 LSTM 셀

LSTM은 그레이디언트 소실 문제를 극복하기 위해 처음 소개되었다.

기본 요소는 RNN의 은닉층을 표현 또는 대체하는 메모리 셀입니다.

이전에 언급했듯이 소실과 폭주 문제를 해결하기 위해 각 메모리 셀에 적절한 가중치 w=1을 유지하는 순환 에지가 있다.

이 순환 에지의 출력을 셀 상태라고 한다.

In [14]:
Image(url='https://git.io/JLdVR', width=700)

-삭제 게이트 : 메모리 셀이 무한정 성장하지 않도록 셀 상태를 다시 설정. 삭제 게이트가 통과할 정보와 억제할 정보를 결정한다.

-입력 게이트와 후보값 : 셀 상태를 업데이트 하는 역할

-출력 게이트 : 은닉 유닛의 출력 값 업데이트

다른 고급 RNN 모델

LSTM은 의존성이 긴 시퀀스를 모델링하는 기본적인 방법을 제공한다.

하지만 많은 변종 있다.

GRU라는 새로운 방식이 소개되었는데 계싼 효율성이 더 높다.

# 16.3 텐서플로로 시퀀스 모델링을 위한 RNN 구현

## 16.3.1 영화 리뷰의 감성 분석

In [15]:
!mkdir ../ch08
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz -O ../ch08/movie_data.csv.gz

mkdir: cannot create directory ‘../ch08’: File exists
--2022-01-07 05:59:26--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz [following]
--2022-01-07 05:59:26--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26521894 (25M) [application/octet-stream]
Saving to: ‘../ch08/movie_data.csv.gz’

../ch

In [16]:
import os
import gzip
import shutil


with gzip.open('../ch08/movie_data.csv.gz', 'rb') as f_in, open('movie_data.csv', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
df = pd.read_csv('movie_data.csv', encoding='utf-8')

df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [17]:
#1단계 : 데이터셋 만들기
target=df.pop('sentiment')
ds_raw=tf.data.Dataset.from_tensor_slices(
    (df.values,target.values)
)

#확인
for ex in ds_raw.take(3):
  tf.print(ex[0].numpy()[0][:50],ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


In [18]:
#훈련, 테스트, 검증 데이터셋으로 나누기
tf.random.set_seed(1)

ds_raw=ds_raw.shuffle(50000,reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [19]:
#신경망의 입력으로 데이터를 준비하기 위해 숫자 값으로 인코딩해야 한다.
## 단계 2: 고유 토큰(단어) 찾기
from collections import Counter
tokenizer=tfds.deprecated.text.Tokenizer()

token_counts=Counter()
for example in ds_raw_train:
  tokens=tokenizer.tokenize(example[0].numpy()[0])
  token_counts.update(tokens)

print('어휘 사전 크기:',len(token_counts))

어휘 사전 크기: 87007


In [20]:
#각가의 단어를 고유 정수로 매핑하기
#키는 고유 단어이고 키에 매핑된 값은 고유한 정수이다.

## 단계 3: 고유 토큰을 정수로 인코딩하기
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)

example_str = 'This is an example!'
encoder.encode(example_str)

[232, 9, 270, 1123]

검증 데이터와 테스트 데이터에 있는 토근이 훈련 데이터에 없다면 매핑되지 않을 수 있다.

q개의 토큰이 있고 이전에 본 적이 없으며 token_counts에 포함되지 않은 모든 토큰은 정수 q+1에 할당된다.

데이터셋 객체에 map 메서드를 사용해 다른 변환을 적용하듯이 데이터셋에 있는 각 텍스트를 변환할 수 있다.

하지만 텍스트 데이터가 텐서 객체에 들어 있어 즉시 실행 모드에서 텐서의 numpy 메소드로 참조할 수 있다. 

하지만 map 메서드로 변환하는 동안에는 즉시 실행이 비활성화된다.

이 문제를 해결하기 위해 두 개의 함수를 정의한다.

첫 번째 함수는 즉시 실행 모드가 활성화된 것처럼 입력 텐서를 다룬다.

In [21]:
#3-A : 변환을 위한 함수 정의
def encode(text_tensor,label):
  text=text_tensor.numpy()[0]
  encoded_text=encoder.encode(text)
  return encoded_text,label

두 번째 함수는 첫 번째 함수를 tf.py_function 으로 감싸서 map 메서드에서 사용할 수 있는 텐서플로 연산으로 변환하겠다.

텍스트를 정수 리스트로 인코딩한느 과정은 다음 코드를 통해 수행된다.



In [22]:
## 단계 3-B: 인코딩 함수를 텐서플로 연산으로 감싸기
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label], 
                          Tout=(tf.int64, tf.int64))

In [23]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
    print('시퀀스 길이:', example[0].shape)
    
example

시퀀스 길이: (24,)
시퀀스 길이: (179,)
시퀀스 길이: (262,)
시퀀스 길이: (535,)
시퀀스 길이: (130,)


(<tf.Tensor: shape=(130,), dtype=int64, numpy=
 array([  579,  1296,    32,   425,    40,   763,  9267,    65,   280,
          308,     6,   481,   155,   473,     2,     3,   684,     9,
          781,   176,   959,   730,  3917,    67,  9905,    13,   277,
           24,    35,   371, 16368,     6,    14, 17231,    29,   187,
         1651,   489,   503,   480,   143,    32,   270,  5851,  2402,
           13,  3592,  3443,   425,  3313,   256,   257,  1577,   117,
            8,   698,   270,   564,    56,     8,    42,  7517,  2629,
          820,    25,    60,    79,   343,    32,   645,    14,   528,
          241,    32,  1980,     8,    56,     8,    42,  1364,   573,
         5183,    43,    12,  3870,    32,   312,   642,   251,  1401,
        17232,     8,   698,   257,   750,     2,     9,    76,   235,
            8,    42,   235,   840,   666,   258, 17233,   419,    32,
        17234,   585,   420,   840,    25,    40,    13,    14,   198,
          266,   623,   173,  

지금까지 단어 시퀀스를 정수 시퀀스로 변환했다.

하지만 시퀀스의 길이가 다르다.

RNN은 다른 길이의 시퀀스를 다룰 수 있지만 미니 배치에 있는 시퀀스는 효율적으로 텐서에 저장하기 위해 동일한 길이가 되어야 한다.

크기가 다른 원소를 가진 데이터셋을 미니 배치로 나누기 위해 텐서플론느 padded_batch 를 사용한다.

하나의 배치에 포함되는 모든 원소를 자동으로 0으로 패딩하여 배치에 있는 모든 시퀀스가 동일한 길이가 되도록 만든다.



In [24]:
## 일부 데이터 추출하기
ds_subset = ds_train.take(8)
for example in ds_subset:
    print('개별 샘플 크기:', example[0].shape)

## 배치 데이터 만들기
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], []))

for batch in ds_batched:
    print('배치 차원:', batch[0].shape)

개별 샘플 크기: (119,)
개별 샘플 크기: (688,)
개별 샘플 크기: (308,)
개별 샘플 크기: (204,)
개별 샘플 크기: (326,)
개별 샘플 크기: (240,)
개별 샘플 크기: (127,)
개별 샘플 크기: (453,)
배치 차원: (4, 688)
배치 차원: (4, 453)


In [25]:
## 배치 데이터 만들기
train_data = ds_train.padded_batch(
    32, padded_shapes=([-1],[]))

valid_data = ds_valid.padded_batch(
    32, padded_shapes=([-1],[]))

test_data = ds_test.padded_batch(
    32, padded_shapes=([-1],[]))

문장 인코딩을 위한 임베딩 층

이전의 준비 단계에서 동일한 길이의 시퀀스를 생성했다.

이 시퀀스의 원소는 고유한 단어의 인덱스에 해당하는 정수이다.

이런 것을 입력 특성으로 변환하는 방법이 있다.

원핫 인코딩.

차원이 굉장히 많아져 차원의 저주 영향을 받을 수 있고 특성 벡터가 매우 희소해진다.

각 단어를 실수값을 가진 고정된 길이의 벡터로 변환하는 것.

임베딩이라고 하는 특성 학습 기법을 사용해 데이터셋에 있는 단어를 표현하는 데 중요한 특성을 자동적으로 학습할 수 있다.

장점은 다음과 같다

- 차원의 저주 영향 감소

- 신경망에서 임베딩 층이 최적화되기에 중요한 특성이 추출

In [26]:
Image(url='https://git.io/JLdV0', width=700)

In [27]:
from tensorflow.keras.layers import Embedding


model = tf.keras.Sequential()

model.add(Embedding(input_dim=100,
                    output_dim=6,
                    input_length=20,
                    name='embed-layer'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


입력은 랭크 2여야 한다.

input_length-> 행길이

output_dim->임베딩 특성 크기

RNN 모델 만들기

-SImple RNN

-LSTM

-GRU

In [28]:
#다층 RNN 모델 만들기
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


감성 분석 작업을 위한 RNN 만들기.

시퀀스 길이가 길기에 LSTM 층을 사용

Bidirectional 래퍼 클래스로 LSTM 층을 감싸겠다.

In [29]:
embedding_dim = 20
vocab_size = len(token_counts) + 2

tf.random.set_seed(1)

## 모델 생성
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),
    
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'), 

    tf.keras.layers.Dense(64, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

## 컴파일과 훈련:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

## 테스트 데이터에서 평가
test_results= bi_lstm_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(test_results[1]*100))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1740180   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 551s 875ms/step - loss: 0.5249 - accuracy: 0.7246 - val_loss: 0.3767 - val_accuracy: 0.8434
Epoch 2/10
625/625 [==========

짧은 시퀀스에 RNN 추가하기

In [30]:
def preprocess_datasets(
    ds_raw_train, 
    ds_raw_valid, 
    ds_raw_test,
    max_seq_length=None,
    batch_size=32):
    
    ## 단계 1: (데이터셋 만들기 이미 완료)
    ## 단계 2: 고유 토큰 찾기
    tokenizer = tfds.deprecated.text.Tokenizer()
    token_counts = Counter()

    for example in ds_raw_train:
        tokens = tokenizer.tokenize(example[0].numpy()[0])
        if max_seq_length is not None:
            tokens = tokens[-max_seq_length:]
        token_counts.update(tokens)

    print('어휘 사전 크기:', len(token_counts))


    ## 단계 3: 텍스트 인코딩하기
    encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
    def encode(text_tensor, label):
        text = text_tensor.numpy()[0]
        encoded_text = encoder.encode(text)
        if max_seq_length is not None:
            encoded_text = encoded_text[-max_seq_length:]
        return encoded_text, label

    def encode_map_fn(text, label):
        return tf.py_function(encode, inp=[text, label], 
                              Tout=(tf.int64, tf.int64))

    ds_train = ds_raw_train.map(encode_map_fn)
    ds_valid = ds_raw_valid.map(encode_map_fn)
    ds_test = ds_raw_test.map(encode_map_fn)

    ## 단계 4: 배치 데이터 만들기
    train_data = ds_train.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    valid_data = ds_valid.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    test_data = ds_test.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    return (train_data, valid_data, 
            test_data, len(token_counts))

In [31]:
def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):

    tf.random.set_seed(1)

    # 모델 생성
    model = tf.keras.Sequential()
    
    model.add(
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            name='embed-layer')
    )
    
    for i in range(n_recurrent_layers):
        return_sequences = (i < n_recurrent_layers-1)
            
        if recurrent_type == 'SimpleRNN':
            recurrent_layer = SimpleRNN(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='simprnn-layer-{}'.format(i))
        elif recurrent_type == 'LSTM':
            recurrent_layer = LSTM(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='lstm-layer-{}'.format(i))
        elif recurrent_type == 'GRU':
            recurrent_layer = GRU(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='gru-layer-{}'.format(i))
        
        if bidirectional:
            recurrent_layer = Bidirectional(
                recurrent_layer, name='bidir-'+recurrent_layer.name)
            
        model.add(recurrent_layer)

    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

In [32]:
from tensorflow.keras.layers import Bidirectional


batch_size = 32
embedding_dim = 20
max_seq_length = 100

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test, 
    max_seq_length=max_seq_length, 
    batch_size=batch_size
)


vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN', 
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

rnn_model.summary()

어휘 사전 크기: 58063
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1161300   
                                                                 
 bidir-simprnn-layer-0 (Bidi  (None, 128)              10880     
 rectional)                                                      
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,180,501
Trainable params: 1,180,501
Non-trainable params: 0
_________________________________________________________________


In [33]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])


history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

Epoch 1/10
625/625 [==============================] - 57s 89ms/step - loss: 0.6922 - accuracy: 0.5249 - val_loss: 0.6670 - val_accuracy: 0.6100
Epoch 2/10
625/625 [==============================] - 56s 89ms/step - loss: 0.5230 - accuracy: 0.7372 - val_loss: 0.6517 - val_accuracy: 0.6186
Epoch 3/10
625/625 [==============================] - 56s 89ms/step - loss: 0.4099 - accuracy: 0.8141 - val_loss: 0.4644 - val_accuracy: 0.8042
Epoch 4/10
625/625 [==============================] - 57s 91ms/step - loss: 0.2376 - accuracy: 0.9092 - val_loss: 0.5451 - val_accuracy: 0.7900
Epoch 5/10
625/625 [==============================] - 54s 86ms/step - loss: 0.1328 - accuracy: 0.9523 - val_loss: 0.8463 - val_accuracy: 0.7054
Epoch 6/10
625/625 [==============================] - 56s 90ms/step - loss: 0.0711 - accuracy: 0.9764 - val_loss: 0.9284 - val_accuracy: 0.7370
Epoch 7/10
625/625 [==============================] - 53s 84ms/step - loss: 0.0804 - accuracy: 0.9724 - val_loss: 1.1674 - val_accuracy:

In [35]:
results = rnn_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(results[1]*100))

782/782 [==============================] - 26s 33ms/step - loss: 1.1059 - accuracy: 0.7482
테스트 정확도: 74.82%
